In [1]:
import pyomo.environ as pyo
import idaes
from idaes.core.solvers import use_idaes_solver_configuration_defaults
import idaes.core.util.scaling as iscale
import ngcc

In [2]:
use_idaes_solver_configuration_defaults()
idaes.cfg.ipopt.options.nlp_scaling_method = "user-scaling"
solver = pyo.SolverFactory("ipopt")

In [3]:
m = pyo.ConcreteModel()
m.fs = ngcc.NgccFlowsheet(default={"dynamic":False})
iscale.calculate_scaling_factors(m)
m.fs.initialize()

2022-02-24 22:40:00 [INFO] idaes.init.fs.fg_translate.properties_in: Starting initialization
2022-02-24 22:40:00 [INFO] idaes.init.fs.fg_translate.properties_in: Property initialization: optimal - Optimal Solution Found.
2022-02-24 22:40:00 [INFO] idaes.init.fs.fg_translate.properties_out: Initialisation Complete, skipped.
2022-02-24 22:40:00 [INFO] idaes.init.fs.fg_translate.properties_out: fs.fg_translate.properties_out State Released.
2022-02-24 22:40:00 [INFO] idaes.init.fs.fg_translate: Initialization Complete optimal - Optimal Solution Found.
Ipopt 3.13.2: nlp_scaling_method=user-scaling
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://gi

In [4]:
res = solver.solve(m, tee=True)

Ipopt 3.13.2: nlp_scaling_method=user-scaling
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale s

In [5]:
jac, nlp = iscale.get_jacobian(m, scaled=True)
print("Extreme Jacobian entries:")
for i in iscale.extreme_jacobian_entries(jac=jac, nlp=nlp, large=100):
    print(f"    {i[0]:.2e}, [{i[1]}, {i[2]}]")
print("Unscaled constraints:")
for c in iscale.unscaled_constraints_generator(m):
    print(f"    {c}")
print("Scaled constraints by factor:")
for c, s in iscale.constraints_with_scale_factor_generator(m):
    print(f"    {c}, {s}")
print("Badly scaled variables:")
for v, sv in iscale.badly_scaled_var_generator(m, large=1e2, small=1e-2, zero=1e-12):
    print(f"    {v} -- {sv} -- {iscale.get_scaling_factor(v)}")
print(f"Jacobian Condition Number: {iscale.jacobian_cond(jac=jac):.2e}")

Extreme Jacobian entries:
    2.49e-05, [fs.gt.cmp1.isentropic[0.0], fs.gt.cmp1.control_volume.properties_in[0.0].log_mole_frac_phase_comp[Vap,CO2]]
    2.49e-05, [fs.gt.cmp1.isentropic[0.0], fs.gt.cmp1.properties_isentropic[0.0].log_mole_frac_phase_comp[Vap,CO2]]
    9.41e-05, [fs.gt.cmp1.isentropic_energy_balance[0.0], fs.gt.cmp1.properties_isentropic[0.0].pressure]
    7.15e-06, [fs.gt.cmp1.control_volume.enthalpy_balances[0.0], fs.gt.cmp1.control_volume.properties_out[0.0].pressure]
    1.33e-05, [fs.gt.splt1.material_splitting_eqn[0.0,air05,CO2], fs.gt.splt1.split_fraction[0.0,air05]]
    1.90e-05, [fs.gt.splt1.material_splitting_eqn[0.0,air05,CO2], fs.gt.splt1.mixed_state[0.0].flow_mol_phase[Vap]]
    1.33e-05, [fs.gt.splt1.material_splitting_eqn[0.0,air07,CO2], fs.gt.splt1.split_fraction[0.0,air07]]
    4.17e-06, [fs.gt.splt1.material_splitting_eqn[0.0,air07,CO2], fs.gt.splt1.mixed_state[0.0].flow_mol_phase[Vap]]
    6.15e-05, [fs.gt.splt1.material_splitting_eqn[0.0,air07,CO2], 

In [6]:
flue_species={ # species set and initial guess
    "O2": 0.1172,
    "H2O": 0.0900,
    "CO2": 0.0421,
    "N2": 0.7419,
    "Ar": 0.0088,
}
sum(x for x in flue_species.values())

1.0

In [7]:
m.fs.hrsg.splitter_ip1.toNGPH.display()

toNGPH : Size=1
    Key  : Name     : Value
    None : enth_mol : {0.0: 17862.287168364004}
         : flow_mol : {0.0: 895.315271965969}
         : pressure : {0.0: 4206000.0}


In [8]:
m.fs.ng_preheat.shell_inlet.display()

shell_inlet : Size=1
    Key  : Name     : Value
    None : enth_mol : {0.0: 17862.287168364004}
         : flow_mol : {0.0: 895.315271965969}
         : pressure : {0.0: 4206000.0}
